In [1]:
import pandas as pd
from statistics import median, mode
#from wrapper import get_bals, acc_info, txsigs_from_address, confirmed_tx_info
from datetime import date
import math

In [2]:
from solana.rpc.api import Client
from solana.rpc.types import TokenAccountOpts

solana_client = Client("https://explorer-api.mainnet-beta.solana.com/")

In [ ]:
def csvdict(paths):
    csvs = {}
    for path in paths:
        name = path.split('.')[-2].split('/')[-1]
        df = pd.read_csv(path)
        csvs[name] = df
        
    return csvs

In [ ]:
def preformat(df):
    col1 = df.columns[0]
    col2 = df.columns[1]
    df.rename(columns = {col1:'pubkey', col2:'nft_count'}, inplace = True)
    df.loc[-1] = [col1, float(col2)]
    df.index = df.index + 1
    df = df.sort_index()
    
    return df

In [ ]:
def add_lamports(df):
    df['lamports'] = df.pubkey.apply(get_bals)
    
    return df

In [ ]:
# metrics on NFT posession count over all users 
def nft_info(df):
    nfts_total = df.nft_count.sum()
    nfts_avg = df.nft_count.mean()
    
    return nfts_avg, nfts_total

In [ ]:
# metrics on sol/lamport balance over all users
def bal_info(df):
    sol_total = round(df.lamports.sum() * 10**-9, 4)
    sol_avg = round(df.lamports.mean() * 10**-9, 4)
    
    return sol_avg, sol_total

In [ ]:
def analyze(csvs):
    results = {}
    for key in list(csvs.keys()):
        df = csvs[key]
        date = pd.to_datetime('today').strftime("%Y/%m/%d")
        no_lamports = preformat(df)
        populated_df = add_lamports(no_lamports)
        nfts_avg, nfts_total = nft_info(populated_df)
        sol_avg, sol_total = bal_info(populated_df)
        
        analytics_dict = {
            'community':[key],
            'date':[date],
            'users':[len(df)],
            'sol_avg':[sol_avg],
            'sol_total':[sol_total],
            'nfts_avg':[nfts_avg], 
            'nfts_total':[nfts_total]
        }
    
        analytics_df = pd.DataFrame.from_dict(analytics_dict)
        analytics_df = analytics_df.set_index('community')
        result = [populated_df, analytics_df]
        results[key] = result
        
    return results

In [ ]:
def scores(csvs):
    for key in list(csvs.keys()):
        df = csvs[key][1]
        users = df['users'].values[0]
        nftnum = df['nfts_total'].values[0]
        while True:
            print("For community: ", key)

            holders = input("Number Of Holders: ")
            try:
                int(holders)
                break
            except:
                print("Invalid entry. Passing NaN.")
                holders = math.nan
                break

            supply = input("Number of NFT Supply: ")
            try:
                int(supply)
                break
            except:
                print("Invalid entry. Passing NaN.")
                supply = math.nan
                break

            floor = input("Floor price: ")
            try:
                int(floor)
                break
            except:
                print("Invalid entry. Passing NaN.")
                floor = math.nan
                break

        user_gscore = float(users) / float(holders)
        nft_gscore = float(nftnum) / float(supply)
        nft_floor = float(floor) * float(supply)

        df['user_grape_score'] = str(round(user_gscore * 100, 2)) + '%'
        df['nft_grape_score']  = str(round(nft_gscore * 100, 2)) + '%'
        df['nft_floor']  = str(round(nft_floor, 3)) + ' SOL'


    return df

In [ ]:
def leaderboard(csvs):
    usernum = {}
    solavg = {}
    soltotal = {}
    nftsavg = {}
    nftstotal = {}
    nftgrapescore = {}
    usergrapescore = {}
    floorprice = {}
    for key in list(csvs.keys()):
        df = csvs[key][1]        
        name = df.index.values[0]
        usernum[name] = df['users'].values[0]
        solavg[name] = df['sol_avg'].values[0]
        soltotal[name] = df['sol_total'].values[0]
        nftsavg[name] = df['nfts_avg'].values[0]
        nftstotal[name] = df['nfts_total'].values[0]
        nftgrapescore[name] = df['nft_grape_score'].values[0]
        usergrapescore[name] = df['user_grape_score'].values[0]
        floorprice[name] = df['nft_floor'].values[0]
        
    userdf = pd.DataFrame(usernum, index=['Users'])
    solavgdf = pd.DataFrame(solavg, index=['Sol Average'])
    totsoldf = pd.DataFrame(soltotal, index=['Sol Total'])
    nftsavgdf = pd.DataFrame(nftsavg, index=['NFTs Average'])
    totnftsdf = pd.DataFrame(nftstotal, index=['NFTs Total'])
    nftgrapescoredf = pd.DataFrame(nftgrapescore, index=['NFT Grape Score'])
    usergrapescoredf = pd.DataFrame(usergrapescore, index=['User Grape Score'])

    return [userdf, solavgdf, totsoldf, nftsavgdf, totnftsdf, nftgrapescoredf, usergrapescoredf]

In [ ]:
def leaderboard_save(csvs):
    for csv in csvs:
        filename = '../metrics/leaderboards/' + csv.index.values[0] + '.csv'
        csv.to_csv(filename)

In [ ]:
def postanalysis_save(csvs):
    for key in list(csvs.keys()):
        csv = csvs[key]
        csv.items
        for df in csv:
            # to be continued

In [ ]:
import os
options_paths = [os.path.join(dp, f) for dp, dn, filenames in os.walk('../csvs/') for f in filenames if os.path.splitext(f)[1] == '.csv'][:5]

In [ ]:
csvs_preanalysis = csvdict(options_paths)
csvs_postanalysis = analyze(csvs_preanalysis)
scores(csvs_postanalysis)
csvs_leaderboard = leaderboard(csvs_postanalysis)
leaderboard_save(csvs_leaderboard)

In [3]:
kaiju = pd.read_csv('kaiju-mintlist.csv')

In [4]:
def preformat_mintlist(df):
    col1 = df.columns[0]
    df.rename(columns = {col1:'pubkey'}, inplace = True)
    df.loc[-1] = col1
    df.index = df.index + 1
    df = df.sort_index()
    
    return df

In [16]:
from time import sleep

In [17]:
sleep(1)

In [36]:
def getowners(pubkey):
    sleep(1)
    address = solana_client.get_token_largest_accounts(pubkey).get('result').get('value')[0].get('address')
    owner = solana_client.get_account_info(address, encoding='jsonParsed').get('result').get('value').get('data').get('parsed').get('info').get('owner')
    return owner

In [7]:
kaiju = preformat_mintlist(kaiju)

In [43]:
len(owners)

5501

In [40]:
uniqueholders = list(set(owners))

In [42]:
len(uniqueholders)

1863

In [44]:
holddf = pd.DataFrame (uniqueholders, columns = ['Unique Holders'])

In [49]:
holddf.to_csv('kaiju-holders.csv')

In [47]:
duplicate = holddf[holddf.duplicated()]

In [48]:
duplicate

,Unique Holders


In [38]:
owners = [getowners(val) for val in tqdm(kaiju.pubkey.values)]



  0%|          | 0/5501 [00:00<?, ?it/s]

  0%|          | 1/5501 [00:01<2:18:55,  1.52s/it]

  0%|          | 2/5501 [00:03<2:19:10,  1.52s/it]

  0%|          | 3/5501 [00:04<2:19:27,  1.52s/it]

  0%|          | 4/5501 [00:06<2:19:14,  1.52s/it]

  0%|          | 5/5501 [00:07<2:19:04,  1.52s/it]

  0%|          | 6/5501 [00:09<2:19:07,  1.52s/it]

  0%|          | 7/5501 [00:10<2:20:03,  1.53s/it]

  0%|          | 8/5501 [00:12<2:19:26,  1.52s/it]

  0%|          | 9/5501 [00:13<2:19:26,  1.52s/it]

  0%|          | 10/5501 [00:15<2:19:16,  1.52s/it]

  0%|          | 11/5501 [00:16<2:19:38,  1.53s/it]

  0%|          | 12/5501 [00:18<2:19:38,  1.53s/it]

  0%|          | 13/5501 [00:19<2:20:03,  1.53s/it]

  0%|          | 14/5501 [00:21<2:20:14,  1.53s/it]

  0%|          | 15/5501 [00:22<2:19:53,  1.53s/it]

  0%|          | 16/5501 [00:24<2:19:37,  1.53s/it]

  0%|          | 17/5501 [00:25<2:19:36,  1.53s/it]

  0%|          | 18/5501 [00:27<2:18:39,  1.52s/it]

  0%|      

  3%|▎         | 138/5501 [03:36<2:17:46,  1.54s/it]

  3%|▎         | 139/5501 [03:37<2:17:51,  1.54s/it]

  3%|▎         | 140/5501 [03:39<2:17:43,  1.54s/it]

  3%|▎         | 141/5501 [03:40<2:16:27,  1.53s/it]

  3%|▎         | 142/5501 [03:42<2:16:31,  1.53s/it]

  3%|▎         | 143/5501 [03:43<2:16:41,  1.53s/it]

  3%|▎         | 144/5501 [03:45<2:16:44,  1.53s/it]

  3%|▎         | 145/5501 [03:46<2:16:23,  1.53s/it]

  3%|▎         | 146/5501 [03:48<2:16:13,  1.53s/it]

  3%|▎         | 147/5501 [03:49<2:15:54,  1.52s/it]

  3%|▎         | 148/5501 [03:51<2:15:59,  1.52s/it]

  3%|▎         | 149/5501 [03:52<2:17:30,  1.54s/it]

  3%|▎         | 150/5501 [03:54<2:17:11,  1.54s/it]

  3%|▎         | 151/5501 [03:56<2:16:47,  1.53s/it]

  3%|▎         | 152/5501 [03:57<2:16:02,  1.53s/it]

  3%|▎         | 153/5501 [03:59<2:16:03,  1.53s/it]

  3%|▎         | 154/5501 [04:00<2:15:44,  1.52s/it]

  3%|▎         | 155/5501 [04:02<2:15:30,  1.52s/it]

  3%|▎         | 156/5501 [0

  5%|▍         | 274/5501 [07:06<2:17:13,  1.58s/it]

  5%|▍         | 275/5501 [07:08<2:16:58,  1.57s/it]

  5%|▌         | 276/5501 [07:09<2:16:59,  1.57s/it]

  5%|▌         | 277/5501 [07:11<2:17:27,  1.58s/it]

  5%|▌         | 278/5501 [07:12<2:17:53,  1.58s/it]

  5%|▌         | 279/5501 [07:14<2:17:51,  1.58s/it]

  5%|▌         | 280/5501 [07:16<2:18:02,  1.59s/it]

  5%|▌         | 281/5501 [07:17<2:18:40,  1.59s/it]

  5%|▌         | 282/5501 [07:19<2:18:23,  1.59s/it]

  5%|▌         | 283/5501 [07:20<2:18:01,  1.59s/it]

  5%|▌         | 284/5501 [07:22<2:18:08,  1.59s/it]

  5%|▌         | 285/5501 [07:24<2:29:58,  1.73s/it]

  5%|▌         | 286/5501 [07:26<2:44:21,  1.89s/it]

  5%|▌         | 287/5501 [07:28<2:48:02,  1.93s/it]

  5%|▌         | 288/5501 [07:30<2:36:38,  1.80s/it]

  5%|▌         | 289/5501 [07:31<2:30:13,  1.73s/it]

  5%|▌         | 290/5501 [07:33<2:24:45,  1.67s/it]

  5%|▌         | 291/5501 [07:34<2:21:30,  1.63s/it]

  5%|▌         | 292/5501 [0

  7%|▋         | 410/5501 [10:37<2:14:33,  1.59s/it]

  7%|▋         | 411/5501 [10:39<2:14:50,  1.59s/it]

  7%|▋         | 412/5501 [10:40<2:14:55,  1.59s/it]

  8%|▊         | 413/5501 [10:42<2:14:41,  1.59s/it]

  8%|▊         | 414/5501 [10:44<2:14:08,  1.58s/it]

  8%|▊         | 415/5501 [10:45<2:14:46,  1.59s/it]

  8%|▊         | 416/5501 [10:47<2:14:22,  1.59s/it]

  8%|▊         | 417/5501 [10:48<2:14:34,  1.59s/it]

  8%|▊         | 418/5501 [10:50<2:15:20,  1.60s/it]

  8%|▊         | 419/5501 [10:52<2:14:51,  1.59s/it]

  8%|▊         | 420/5501 [10:53<2:15:10,  1.60s/it]

  8%|▊         | 421/5501 [10:55<2:14:45,  1.59s/it]

  8%|▊         | 422/5501 [10:56<2:17:36,  1.63s/it]

  8%|▊         | 423/5501 [10:58<2:16:10,  1.61s/it]

  8%|▊         | 424/5501 [11:00<2:17:08,  1.62s/it]

  8%|▊         | 425/5501 [11:01<2:17:42,  1.63s/it]

  8%|▊         | 426/5501 [11:03<2:17:51,  1.63s/it]

  8%|▊         | 427/5501 [11:04<2:15:58,  1.61s/it]

  8%|▊         | 428/5501 [1

 10%|▉         | 546/5501 [14:11<2:07:07,  1.54s/it]

 10%|▉         | 547/5501 [14:12<2:06:29,  1.53s/it]

 10%|▉         | 548/5501 [14:14<2:05:34,  1.52s/it]

 10%|▉         | 549/5501 [14:15<2:05:48,  1.52s/it]

 10%|▉         | 550/5501 [14:17<2:07:04,  1.54s/it]

 10%|█         | 551/5501 [14:18<2:06:38,  1.54s/it]

 10%|█         | 552/5501 [14:20<2:06:25,  1.53s/it]

 10%|█         | 553/5501 [14:22<2:07:11,  1.54s/it]

 10%|█         | 554/5501 [14:23<2:06:16,  1.53s/it]

 10%|█         | 555/5501 [14:25<2:05:43,  1.53s/it]

 10%|█         | 556/5501 [14:26<2:06:11,  1.53s/it]

 10%|█         | 557/5501 [14:28<2:06:06,  1.53s/it]

 10%|█         | 558/5501 [14:29<2:06:06,  1.53s/it]

 10%|█         | 559/5501 [14:31<2:06:49,  1.54s/it]

 10%|█         | 560/5501 [14:32<2:07:26,  1.55s/it]

 10%|█         | 561/5501 [14:34<2:06:46,  1.54s/it]

 10%|█         | 562/5501 [14:35<2:06:40,  1.54s/it]

 10%|█         | 563/5501 [14:37<2:05:59,  1.53s/it]

 10%|█         | 564/5501 [1

...

 90%|████████▉ | 4937/5501 [2:16:18<14:20,  1.53s/it]

 90%|████████▉ | 4938/5501 [2:16:19<14:25,  1.54s/it]

 90%|████████▉ | 4939/5501 [2:16:21<14:22,  1.53s/it]

 90%|████████▉ | 4940/5501 [2:16:22<14:19,  1.53s/it]

 90%|████████▉ | 4941/5501 [2:16:24<14:15,  1.53s/it]

 90%|████████▉ | 4942/5501 [2:16:25<14:12,  1.53s/it]

 90%|████████▉ | 4943/5501 [2:16:27<14:29,  1.56s/it]

 90%|████████▉ | 4944/5501 [2:16:28<14:39,  1.58s/it]

 90%|████████▉ | 4945/5501 [2:16:30<14:42,  1.59s/it]

 90%|████████▉ | 4946/5501 [2:16:32<14:44,  1.59s/it]

 90%|████████▉ | 4947/5501 [2:16:33<14:45,  1.60s/it]

 90%|████████▉ | 4948/5501 [2:16:35<14:45,  1.60s/it]

 90%|████████▉ | 4949/5501 [2:16:36<14:41,  1.60s/it]

 90%|████████▉ | 4950/5501 [2:16:38<14:42,  1.60s/it]

 90%|█████████ | 4951/5501 [2:16:40<14:45,  1.61s/it]

 90%|█████████ | 4952/5501 [2:16:41<14:41,  1.61s/it]

 90%|█████████ | 4953/5501 [2:16:43<14:43,  1.61s/it]

 90%|█████████ | 4954/5501 [2:16:45<14:43,  1.62s/it]

 90%|█████

 92%|█████████▏| 5071/5501 [2:19:51<10:58,  1.53s/it]

 92%|█████████▏| 5072/5501 [2:19:52<10:56,  1.53s/it]

 92%|█████████▏| 5073/5501 [2:19:54<10:51,  1.52s/it]

 92%|█████████▏| 5074/5501 [2:19:55<10:50,  1.52s/it]

 92%|█████████▏| 5075/5501 [2:19:57<10:46,  1.52s/it]

 92%|█████████▏| 5076/5501 [2:19:58<10:50,  1.53s/it]

 92%|█████████▏| 5077/5501 [2:20:00<10:47,  1.53s/it]

 92%|█████████▏| 5078/5501 [2:20:01<10:49,  1.53s/it]

 92%|█████████▏| 5079/5501 [2:20:03<10:44,  1.53s/it]

 92%|█████████▏| 5080/5501 [2:20:04<10:44,  1.53s/it]

 92%|█████████▏| 5081/5501 [2:20:06<10:39,  1.52s/it]

 92%|█████████▏| 5082/5501 [2:20:07<10:42,  1.53s/it]

 92%|█████████▏| 5083/5501 [2:20:09<10:40,  1.53s/it]

 92%|█████████▏| 5084/5501 [2:20:10<10:40,  1.54s/it]

 92%|█████████▏| 5085/5501 [2:20:12<10:35,  1.53s/it]

 92%|█████████▏| 5086/5501 [2:20:13<10:37,  1.54s/it]

 92%|█████████▏| 5087/5501 [2:20:15<10:34,  1.53s/it]

 92%|█████████▏| 5088/5501 [2:20:17<10:37,  1.54s/it]

 93%|█████

 95%|█████████▍| 5205/5501 [2:23:25<07:45,  1.57s/it]

 95%|█████████▍| 5206/5501 [2:23:27<07:40,  1.56s/it]

 95%|█████████▍| 5207/5501 [2:23:28<07:33,  1.54s/it]

 95%|█████████▍| 5208/5501 [2:23:30<07:40,  1.57s/it]

 95%|█████████▍| 5209/5501 [2:23:31<07:43,  1.59s/it]

 95%|█████████▍| 5210/5501 [2:23:33<07:44,  1.60s/it]

 95%|█████████▍| 5211/5501 [2:23:35<07:44,  1.60s/it]

 95%|█████████▍| 5212/5501 [2:23:36<07:44,  1.61s/it]

 95%|█████████▍| 5213/5501 [2:23:38<07:42,  1.61s/it]

 95%|█████████▍| 5214/5501 [2:23:39<07:40,  1.60s/it]

 95%|█████████▍| 5215/5501 [2:23:41<07:39,  1.61s/it]

 95%|█████████▍| 5216/5501 [2:23:43<07:39,  1.61s/it]

 95%|█████████▍| 5217/5501 [2:23:44<07:37,  1.61s/it]

 95%|█████████▍| 5218/5501 [2:23:46<07:34,  1.61s/it]

 95%|█████████▍| 5219/5501 [2:23:47<07:34,  1.61s/it]

 95%|█████████▍| 5220/5501 [2:23:49<07:34,  1.62s/it]

 95%|█████████▍| 5221/5501 [2:23:51<07:33,  1.62s/it]

 95%|█████████▍| 5222/5501 [2:23:52<07:30,  1.62s/it]

 95%|█████

 97%|█████████▋| 5339/5501 [2:26:58<04:07,  1.53s/it]

 97%|█████████▋| 5340/5501 [2:27:00<04:06,  1.53s/it]

 97%|█████████▋| 5341/5501 [2:27:02<04:11,  1.57s/it]

 97%|█████████▋| 5342/5501 [2:27:03<04:14,  1.60s/it]

 97%|█████████▋| 5343/5501 [2:27:05<04:15,  1.62s/it]

 97%|█████████▋| 5344/5501 [2:27:07<04:19,  1.65s/it]

 97%|█████████▋| 5345/5501 [2:27:08<04:18,  1.66s/it]

 97%|█████████▋| 5346/5501 [2:27:10<04:16,  1.65s/it]

 97%|█████████▋| 5347/5501 [2:27:12<04:18,  1.68s/it]

 97%|█████████▋| 5348/5501 [2:27:13<04:19,  1.70s/it]

 97%|█████████▋| 5349/5501 [2:27:15<04:15,  1.68s/it]

 97%|█████████▋| 5350/5501 [2:27:17<04:13,  1.68s/it]

 97%|█████████▋| 5351/5501 [2:27:18<04:09,  1.67s/it]

 97%|█████████▋| 5352/5501 [2:27:20<04:08,  1.66s/it]

 97%|█████████▋| 5353/5501 [2:27:22<04:07,  1.67s/it]

 97%|█████████▋| 5354/5501 [2:27:23<04:05,  1.67s/it]

 97%|█████████▋| 5355/5501 [2:27:25<04:03,  1.67s/it]

 97%|█████████▋| 5356/5501 [2:27:27<04:00,  1.66s/it]

 97%|█████

 99%|█████████▉| 5473/5501 [2:30:34<00:45,  1.61s/it]

100%|█████████▉| 5474/5501 [2:30:35<00:44,  1.64s/it]

100%|█████████▉| 5475/5501 [2:30:37<00:44,  1.70s/it]

100%|█████████▉| 5476/5501 [2:30:39<00:43,  1.74s/it]

100%|█████████▉| 5477/5501 [2:30:41<00:41,  1.72s/it]

100%|█████████▉| 5478/5501 [2:30:42<00:40,  1.75s/it]

100%|█████████▉| 5479/5501 [2:30:44<00:38,  1.73s/it]

100%|█████████▉| 5480/5501 [2:30:46<00:36,  1.75s/it]

100%|█████████▉| 5481/5501 [2:30:48<00:34,  1.73s/it]

100%|█████████▉| 5482/5501 [2:30:49<00:32,  1.74s/it]

100%|█████████▉| 5483/5501 [2:30:51<00:31,  1.73s/it]

100%|█████████▉| 5484/5501 [2:30:53<00:31,  1.86s/it]

100%|█████████▉| 5485/5501 [2:30:55<00:31,  1.98s/it]

100%|█████████▉| 5486/5501 [2:30:58<00:30,  2.04s/it]

100%|█████████▉| 5487/5501 [2:31:00<00:27,  1.98s/it]

100%|█████████▉| 5488/5501 [2:31:01<00:24,  1.89s/it]

100%|█████████▉| 5489/5501 [2:31:04<00:25,  2.14s/it]

100%|█████████▉| 5490/5501 [2:31:07<00:25,  2.33s/it]

100%|█████

In [24]:
kaiju['owner'] = kaiju.pubkey.apply(getowners)

KeyboardInterrupt: 

In [ ]:
kaiju

In [12]:
owner = getowners('C5qVSotgQ3WDCcqJwxQAsYrB8rs6GZPQ2xT4qW268Jd6')

In [13]:
owner

'8NQSZ5wpdET6dzAao2Vg13BsiF4QLAFKPSntKAaggbLr'